In [ ]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue
from collections import deque

# Initialize global variables
GRID_SIZE = 20
CANVAS_SIZE = 600
BUTTON_HEIGHT = 50

# Maze sizes for small, medium, and large
SMALL_MAZE = 10
MEDIUM_MAZE = 20
LARGE_MAZE = 30

# 8-Puzzle Goal State
PUZZLE_GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]  # 0 represents the empty tile


class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Search Problem Solver")

        # Create canvas for maze problems
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=6)

        # Add control buttons
        self.random_button = tk.Button(self.root, text="Random Maze", command=self.generate_random_maze)
        self.random_button.grid(row=1, column=0)

        self.random_agent_button = tk.Button(self.root, text="Random Agent", command=self.run_random_agent)
        self.random_agent_button.grid(row=1, column=1)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.solve_with_dfs)
        self.dfs_button.grid(row=1, column=2)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=self.solve_with_bfs)
        self.bfs_button.grid(row=1, column=3)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=self.solve_with_astar)
        self.astar_button.grid(row=1, column=4)

        self.bidirectional_button = tk.Button(self.root, text="Solve with Bidirectional", command=self.solve_with_bidirectional)
        self.bidirectional_button.grid(row=1, column=5)

        self.problem_var = tk.StringVar(value="Maze")
        self.problem_menu = tk.OptionMenu(self.root, self.problem_var, "Maze", "String Transformation", "8-Puzzle",
                                          command=self.change_problem)
        self.problem_menu.grid(row=1, column=6)

        # Initial maze size and maze generation
        self.problem_type = "Maze"
        self.maze_size = SMALL_MAZE
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.explored_path = []
        self.draw_maze()

    def change_problem(self, problem):
        self.problem_type = problem
        if problem == "Maze":
            self.generate_random_maze()
        elif problem == "String Transformation":
            self.generate_string_problem()
        elif problem == "8-Puzzle":
            self.generate_puzzle_problem()

    def generate_random_maze(self):
        """Generates a random maze and redraws it"""
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.explored_path = []
        self.draw_maze()

    def generate_maze(self, size):
        """Generates a solvable maze with random walls."""
        maze = np.ones((size, size), dtype=int)
        start = (0, 0)
        maze[0][0] = 0  # Start point
        maze[size - 1][size - 1] = 0  # End point

        path = []
        visited = set()

        def dfs_generate(x, y):
            path.append((x, y))
            visited.add((x, y))
            maze[x][y] = 0

            if (x, y) == (size - 1, size - 1):
                return True

            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            random.shuffle(directions)

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < size and 0 <= ny < size and (nx, ny) not in visited:
                    if dfs_generate(nx, ny):
                        return True

            return False

        dfs_generate(0, 0)

        for i in range(size):
            for j in range(size):
                if (i, j) not in path and random.random() > 0.3:
                    maze[i][j] = 1  # Wall

        return maze

    def generate_string_problem(self):
        """Sets up a string transformation problem"""
        self.start_word = "cat"
        self.end_word = "dog"
        self.valid_words = set(["cat", "bat", "bot", "bog", "dog", "cog", "cot", "dot"])
        self.solution_path = None
        self.explored_path = []

    def generate_puzzle_problem(self):
        """Sets up an 8-puzzle problem"""
        self.puzzle_start = [1, 2, 3, 4, 5, 6, 0, 7, 8]  # Example start state
        self.puzzle_goal = PUZZLE_GOAL_STATE
        self.solution_path = None
        self.explored_path = []

    def run_random_agent(self):
        """Runs a random agent for the selected problem."""
        if self.problem_type == "Maze":
            self.random_agent_maze()
        elif self.problem_type == "String Transformation":
            self.random_agent_string()
        elif self.problem_type == "8-Puzzle":
            self.random_agent_puzzle()

    def random_agent_maze(self):
        """Random agent for maze."""
        current = (0, 0)
        goal = (len(self.maze) - 1, len(self.maze) - 1)
        path = [current]

        while current != goal:
            neighbors = get_neighbors(current, self.maze)
            if neighbors:
                current = random.choice(neighbors)
                path.append(current)
            else:
                break

        self.solution_path = path
        self.explored_path = path
        self.show_solution(0, "Random Agent (Maze)")

    def random_agent_string(self):
        """Random agent for string transformation."""
        current = self.start_word
        path = [current]

        while current != self.end_word:
            valid_next_words = [word for word in self.valid_words if self.is_one_letter_diff(current, word)]
            if valid_next_words:
                current = random.choice(valid_next_words)
                path.append(current)
            else:
                break

        self.solution_path = path
        print("Random Agent (String):", path)

    def random_agent_puzzle(self):
        """Random agent for 8-puzzle."""
        current = self.puzzle_start[:]
        path = [current]

        for _ in range(50):  # Limit random moves to avoid infinite loops
            neighbors = self.get_puzzle_neighbors(current)
            if neighbors:
                current = random.choice(neighbors)
                path.append(current)
            else:
                break

        self.solution_path = path
        print("Random Agent (8-Puzzle):", path)

    def get_puzzle_neighbors(self, state):
        """Returns neighbors of the puzzle state."""
        neighbors = []
        zero_index = state.index(0)
        swap_positions = [(1, 3), (1, -3), (0, 1), (0, -1)]

        for dx, dy in swap_positions:
            new_index = zero_index + dx + dy * 3
            if 0 <= new_index < 9:
                neighbor = state[:]
                neighbor[zero_index], neighbor[new_index] = neighbor[new_index], neighbor[zero_index]
                neighbors.append(neighbor)

        return neighbors

    def is_one_letter_diff(self, word1, word2):
        """Checks if two words differ by exactly one letter."""
        return sum(c1 != c2 for c1, c2 in zip(word1, word2)) == 1

    def draw_maze(self):
        self.canvas.delete("all")
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                color = 'green' if (i, j) == (0, 0) else 'red' if (i, j) == (size - 1, size - 1) else 'black' if self.maze[i][j] == 1 else 'white'
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill=color)
        self.root.update()


def get_neighbors(pos, maze):
    """Get valid neighbors."""
    row, col = pos
    neighbors = []
    if row > 0 and maze[row - 1][col] == 0:
        neighbors.append((row - 1, col))
    if row < len(maze) - 1 and maze[row + 1][col] == 0:
        neighbors.append((row + 1, col))
    if col > 
